# Projeto 2 Computação Grafica

## Aluno: Rodrigo Rodrigues de Castro (13695362)

### Cena:

Piquenique no quintal de uma casa, com um caracol comendo as flores (rotação) pelo jardim, flores nascendo, enquanto
dentro da casa há uma pessoa andando pela cozinha e sala.

### Objetos:

Internos (Casa - carpete azul):
- Sofá                           
- Televisão                      
- Ser Humano                     [Translação]        
- Geladeira
- Mesa    

Externos (Quintal - grama):     
- Flores                         [Escala]                
- Arvore                        
- Pique Nique                         
    - Mesa, Pão, Cesta           
- Caracol                       [Rotação]          


## Controle por Teclado

- **Tecla ESC**: Fecha a janela.
- **Teclas W, A, S, D**: Movimentam a câmera para frente, esquerda, trás e direita, respectivamente.
- **Teclas de seta (Cima, Baixo, Esquerda, Direita)**: Movem o objeto ("humano") no espaço 3D.
- **Teclas Z e X**: Alteram a escala do objeto ("flor"). Z aumenta a escala e X diminui.
- **Tecla F**: Rotaciona o objeto ("caracol") no sentido horário.
- **Tecla G**: Rotaciona o objeto ("caracol") no sentido anti-horário.
- **Tecla T**: Executa rotação e escala simultaneamente.
- **Tecla P**: Alterna para o modo Wireframe (visualização em malha).
- **Tecla O**: Volta para o modo de visualização preenchida.

# Inicialização as Janelas e Compilação dos Shaders 

In [ ]:
# Python 3.13

import glfw
from OpenGL.GL import *
import numpy as np
import glm
import math
from numpy import random
from PIL import Image

from shaders.shader_s import Shader



#----------------------(Inicia Janelas + Shaders)-------------------------
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

altura = 800
largura = 800

window = glfw.create_window(largura, altura, "Programa", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

ourShader = Shader("shaders/vertex_shader.vs", "shaders/fragment_shader.fs")
ourShader.use()

program = ourShader.getProgram()


libdecor-gtk-WARNING: Failed to initialize GTK
Failed to load plugin 'libdecor-gtk.so': failed to init


# Variáveis Globais

In [2]:

# Parametros da Camera
cameraPos   = glm.vec3(-10.0, 3.0, 30.0)
cameraFront = glm.vec3(0.0, 0.0, -1.0)
cameraUp    = glm.vec3(0.0, 1.0, 0.0)

firstMouse = True
yaw   = -90.0	
pitch =  0.0
lastX =  largura / 2.0
lastY =  altura / 2.0
fov   =  45.0

# Time
deltaTime = 0.0
lastFrame = 0.0

firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

# Coeficiente de escala para as Flores
s_inc = 2

# Coeficiente de rotação para os Caracois
r_inc = 0.0

# Translação do humano
moveSpeed = 0.2
obj_pos_x = -2
obj_pos_z = -5

# Funções para Carregamento Modelos (vértices e texturas) a partir de Arquivos

In [3]:
glEnable(GL_TEXTURE_2D)
glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
glEnable( GL_BLEND )
glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
glEnable(GL_LINE_SMOOTH)

global vertices_list
vertices_list = []    
global textures_coord_list
textures_coord_list = []


# Carrega um modelo 3D no formato Wavefront .obj, extraindo vértices, coordenadas de textura e faces
def load_model_from_file(filename):
    
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    for line in open(filename, "r"):
        if line.startswith('#'): continue 
        values = line.split() 
        if not values: continue

        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model

# Carrega uma textura de imagem para um objeto padrão e associa a um ID de textura no OpenGL
def load_texture_from_file(texture_id, img_textura):
    
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    img = img.convert("RGB")
    
    image_data = img.tobytes("raw", "RGB", 0, -1)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

# Carrega a textura de imagem com canal alfa (RGBA)para o plano de fundo como céu
def load_texture_from_file_sky(texture_id, img_textura):
    
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    img = img.convert("RGB")

    image_data = img.tobytes("raw", "RGB", 0, -1)
    img = img.convert("RGBA")
    image_data = img.tobytes("raw", "RGBA", 0, -1)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)


# Retorna a sequência de vértices para modelos .obj cujas faces não sejam triângulos
def circular_sliding_window_of_three(arr):
    if len(arr) == 3:
        return arr
    circular_arr = arr + [arr[0]]
    result = []
    for i in range(len(circular_arr) - 2):
        result.extend(circular_arr[i:i+3])
    return result


# Carrega um modelo .obj e uma lista de texturas associadas, preenchendo as listas globais de vértices e coordenadas
def load_obj_and_texture(objFile, texturesList):
    modelo = load_model_from_file(objFile)
    
    verticeInicial = len(vertices_list)
    print('Processando modelo {}. Vertice inicial: {}'.format(objFile, len(vertices_list)))
    faces_visited = []
    for face in modelo['faces']:
        if face[2] not in faces_visited:
            faces_visited.append(face[2])
        for vertice_id in circular_sliding_window_of_three(face[0]):
            vertices_list.append(modelo['vertices'][vertice_id - 1])
        for texture_id in circular_sliding_window_of_three(face[1]):
            if texture_id > 0:
                textures_coord_list.append(modelo['texture'][texture_id - 1])
            else:
                textures_coord_list.append([0.0, 0.0])  # Ou outra coordenada padrão

        
    verticeFinal = len(vertices_list)
    print('Processando modelo {}. Vertice final: {}'.format(objFile, len(vertices_list)))
    
    for id in range(len(texturesList)):
        load_texture_from_file(id,texturesList[id])
    
    return verticeInicial, verticeFinal - verticeInicial

# Carrega um modelo .obj e associa múltiplas texturas a diferentes materiais usados no modelo
def load_obj_and_textures_multi(objFile, material_texture_map):
    modelo = load_model_from_file(objFile)

    resultados = []

    faces_by_material = {}
    for face in modelo['faces']:
        material = face[2]
        if material not in faces_by_material:
            faces_by_material[material] = []
        faces_by_material[material].append(face)

    textura_ids = {}

    for idx, (material, faces) in enumerate(faces_by_material.items()):
        texture_path = material_texture_map.get(material)
        if texture_path is None:
            continue

        texture_id = glGenTextures(1)
        load_texture_from_file(texture_id, texture_path)
        textura_ids[material] = texture_id

        vertice_inicial = len(vertices_list)

        for face in faces:
            for vertice_id in circular_sliding_window_of_three(face[0]):
                vertices_list.append(modelo['vertices'][vertice_id - 1])
            for texture_id_coord in circular_sliding_window_of_three(face[1]):
                if texture_id_coord > 0:
                    textures_coord_list.append(modelo['texture'][texture_id_coord - 1])
                else:
                    textures_coord_list.append([0.0, 0.0])

        vertice_final = len(vertices_list)
        resultados.append((vertice_inicial, vertice_final, texture_id))

    return resultados

# Mapeamento de Materiais e Funções de Desenho para Cena 3D Interativa

In [4]:
# Mapeamento das texturas para cada material de cada objeto
material_texture_map = {
    'Drk_Wood': 'obj/sofa/sofa_textures/Color.jpg',
    'Label': 'obj/sofa/sofa_textures/tag.jpg',
    'Orange_Leather_simple': 'obj/sofa/sofa_textures/orange_leather.jpg',
    'UnderMesh': 'obj/sofa/sofa_textures/metallic.png'
}

material_texture_map_flor = {
    'objectSG': 'obj/flower/123.bmp'
}


material_texture_map_fridge = {
    'Galvanized_steel.001': 'obj/refrigerator/back_metal_texture/GalvanizedSteel01_2K_BaseColor.jpg',
    'baked.002': 'obj/refrigerator/color.png',
    'black.002': None  
}

material_texture_map_tv = {
    'V-Tv': 'obj/television/Texture/V-TV_Base_Color.PNG'
}

material_texture_map_table = {
    'Black_Oak_Wood_Veneer': 'obj/table_outside/Texture_maps/c17_wood_02_diff.jpg',
    'Bolt': None,
    'Wood': 'obj/table_outside/Texture_maps/woodcreator_1_basecolor.jpg'
}

material_texture_map_table_in = {
    'Wood(Table)': 'obj/table/wooden_table_color.png',
}

material_texture_map_bread = {
    'bread03': 'obj/bread/bread03DiffuseMap.png',
    'bread1': 'obj/bread/bread01DiffuseMap.png',
    'bread2': 'obj/bread/bread02DiffuseMap.png'
}

material_texture_map_wicker = {
    'Wood': 'obj/basket/Textures/WoodFineDark004_COL_2K.jpg'
}

material_texture_map_tree = {
    'Bark': 'obj/tree/bark.jpg',
    'Leaves': 'obj/tree/green.jpg'
}

material_texture_map_grama = {
    'Material.001': 'obj/grass/grass2.png'
}

material_texture_map_man= {
    'body': 'obj/human/TEXTURES/T_body_A.png',
    'clothes': 'obj/human/TEXTURES/T_clothes_A.png',
    'eyes': 'obj/human/TEXTURES/T_eyes_A.png',
    'head': 'obj/human/TEXTURES/T_head_A.png',
}

material_texture_map_house = {
    'wire_086086086': None,
    '13___Default': 'obj/house/TEX.png'
}

material_texture_map_snail = {
    'mat0': 'obj/snail/Image_0.jpg'
}

# Carregando os Objetos e suas Texturas
resultados_tree = load_obj_and_textures_multi('obj/tree/Tree_obj.obj', material_texture_map_tree)
resultados_sofa = load_obj_and_textures_multi('obj/sofa/sofa.obj', material_texture_map)
resultados_flor = load_obj_and_textures_multi('obj/flower/Rose.obj', material_texture_map_flor)
resultados_tv = load_obj_and_textures_multi('obj/television/V-Tv.obj', material_texture_map_tv)
resultados_table = load_obj_and_textures_multi('obj/table_outside/woodentable.obj', material_texture_map_table)
resultados_bread = load_obj_and_textures_multi('obj/bread/bread.obj', material_texture_map_bread)
resultados_basket = load_obj_and_textures_multi('obj/basket/wicker_basket.obj', material_texture_map_wicker)
resultados_house = load_obj_and_textures_multi('obj/house/1.obj', material_texture_map_house)
resultados_grama = load_obj_and_textures_multi('obj/grass/GRASS.obj', material_texture_map_grama)
resultados_man= load_obj_and_textures_multi('obj/human/man.obj', material_texture_map_man)
resultados_fridge = load_obj_and_textures_multi('obj/refrigerator/fridge_export.obj',material_texture_map_fridge)
resultados_table_in = load_obj_and_textures_multi('obj/table/WoodenTable.obj',material_texture_map_table_in)
resultados_snail =  load_obj_and_textures_multi('obj/snail/snail.obj',material_texture_map_snail)

#Função genérica para desenhar todos os objetos
def desenha_objeto(resultados, t_x=0.0, t_y=0.0, t_z=0.0, 
                   angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                   s_x=1.0, s_y=1.0, s_z=1.0):
    
    global vertices

    # Calcula a matriz model
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z,)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    # Desenha cada parte do objeto
    for vertice_inicial, vertice_final, texture_id in resultados:
        glBindTexture(GL_TEXTURE_2D, texture_id)
        quantos_vertices = vertice_final - vertice_inicial
        glDrawArrays(GL_TRIANGLES, vertice_inicial, quantos_vertices)

# ---------------------------------- Exterior --------------------------------------

def desenha_grama():
    desenha_objeto(resultados_grama,
                   t_x=0.0, t_y= 0.0, t_z=0.0,
                   angle=0.0, r_x=0.0, r_y=0.0, r_z=0.0,
                   s_x=3.5, s_y=0.1, s_z=3.5)

def desenha_house():
    desenha_objeto(resultados_house,
                   t_x=-5.0, t_y=-0.6, t_z=0.0,
                   angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                   s_x=0.1, s_y=0.1, s_z=0.1)

def desenha_snail(r_inc): # Rotação definida
    snails = [
        (-25.0, 0.5, 20.0),
        (25.0, 0.5, 20.0),
        (-25.0, 0.5, -20.0),
        (-4.0, 0.5, -18.0),
        (35.0, 0.5, 5.0),
        (-35.0, 0.5, -5.0),
        (0.0, 0.5, 35.0)
    ]
    
    for t_x, t_y, t_z in snails:
        desenha_objeto(resultados_snail,
                    t_x=t_x, t_y=t_y, t_z=t_z,
                    angle=r_inc, r_x=0.0, r_y=1.0, r_z=0.0,
                    s_x=0.008, s_y=0.008, s_z=0.008)
    

def desenha_tree():
    trees = [
        (-18.0, 0.0, -8.0),     
        (-10.0, 0.0, -30.0),
        (0.0, 0.0, -35.0),
        (10.0, 0.0, -28.0),
        (20.0, 0.0, -32.0),
        (-25.0, 0.0, -40.0),
        (5.0, 0.0, -38.0),
        (-15.0, 0.0, -42.0),
        (-5.0, 0.0, -33.0),
        (15.0, 0.0, -41.0),
        (25.0, 0.0, -39.0),
        (-30.0, 0.0, -44.0),
        (-20.0, 0.0, -37.0),
        (-8.0, 0.0, -31.0),
        (2.0, 0.0, -34.0),
        (12.0, 0.0, -29.0),
        (22.0, 0.0, -36.0),
        (32.0, 0.0, -43.0),
        (18.0, 0.0, -25.0),
        (8.0, 0.0, -27.0),
        (-12.0, 0.0, -26.0),
        (-22.0, 0.0, -29.0),
        (28.0, 0.0, -30.0),
        (-28.0, 0.0, -41.0),
        (0.0, 0.0, -20.0),
        (20.0, 0.0, -22.0),
        (-5.0, 0.0, -24.0),
        (-35.0, 0.0, -33.0),
        (-40.0, 0.0, -28.0),
        (-45.0, 0.0, -35.0),
        (35.0, 0.0, -32.0),
        (40.0, 0.0, -38.0),
        (45.0, 0.0, -44.0),
        (50.0, 0.0, -30.0),
        (-50.0, 0.0, -42.0),
        (30.0, 0.0, -45.0),
    ]
    
    for t_x, t_y, t_z in trees:
        desenha_objeto(resultados_tree,
                       t_x=t_x, t_y=t_y, t_z=t_z,
                       angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                       s_x=1.5, s_y=1.5, s_z=1.5)


def desenha_basket(): 
    desenha_objeto(resultados_basket,
                   t_x=-15.0, t_y=1.89, t_z=14.5,
                   angle=0, r_x=0, r_y=1.0, r_z=0,
                   s_x=0.3, s_y=0.3, s_z=0.3)
    
def desenha_flor(s_inc): # Escala definida
    flores = [
        (-25.0, 0.0, 20.0),
        (25.0, 0.0, 20.0),
        (-25.0, 0.0, -20.0),
        (25.0, 0.0, -20.0),
        (15.0, 0.0, 10.0),
        (-15.0, 0.0, -10.0),
        (30.0, 0.0, 0.0),
        (-30.0, 0.0, 0.0),
        (0.0, 0.0, 25.0),
        (10.0, 0.0, -30.0),
        (-10.0, 0.0, 30.0),
        (20.0, 0.0, -10.0),
        (-20.0, 0.0, 10.0),
        (5.0, 0.0, 15.0),
        (-5.0, 0.0, -15.0),
        (35.0, 0.0, 5.0),
    ]
    
    for t_x, t_y, t_z in flores:
        desenha_objeto(resultados_flor,
                       t_x=t_x, t_y=t_y, t_z=t_z,
                       angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                       s_x=s_inc, s_y=s_inc, s_z=s_inc)
    
def desenha_table():
    desenha_objeto(resultados_table,
                   t_x=-15.0, t_y=0.0, t_z=15.0,
                   angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                   s_x=2.0, s_y=2.0, s_z=2.0)
    
def desenha_bread():
    desenha_objeto(resultados_bread,
                   t_x=-15.0, t_y=1.9, t_z=16.0,
                   angle=0.0, r_x=0.0, r_y=1.0, r_z=0.0,
                   s_x=0.05, s_y=0.05, s_z=0.05)
    
#-------------------------------------- Interior --------------------------------------

def desenha_man(obj_pos_x, obj_pos_z): # Translação definida
    desenha_objeto(resultados_man,
                   t_x=obj_pos_x, t_y=0.0, t_z=obj_pos_z,
                   angle=0.0, r_x=-15.0, r_y=1.0, r_z=0.0,
                   s_x=1.3, s_y=1.3, s_z=1.3)
    

def desenha_sofa():
    desenha_objeto(resultados_sofa,
                   t_x=-3, t_y=0.0, t_z=-4,
                   s_x=0.05, s_y=0.05, s_z=0.05)

def desenha_fridge():
    desenha_objeto(resultados_fridge,
                   t_x=-10.3, t_y=0.0, t_z=-4.4,
                   angle=0, r_x=0, r_y=1.0, r_z=0,
                   s_x=1.3, s_y=1.3, s_z=1.3)

def desenha_tv():
    desenha_objeto(resultados_tv,
                   t_x=-3, t_y=0.1, t_z=-1,
                   angle=180, r_x=0, r_y=1.0, r_z=0,
                   s_x=1.2, s_y=1.2, s_z=1.2)

def desenha_table_in():
    desenha_objeto(resultados_table_in,
                   t_x=-8, t_y=0.2, t_z=-1,
                   angle=90, r_x=0, r_y=1.0, r_z=0,
                   s_x=0.9, s_y=0.9, s_z=0.9)



# Matrizes Model, View e Projection

In [5]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    angle = math.radians(angle)
    matrix_transform = glm.mat4(1.0) 

    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))                   # translacao
    if angle!=0: matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))  # rotação
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))                       # escala

    matrix_transform = np.array(matrix_transform)
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    mat_projection = glm.perspective(glm.radians(fov), largura/altura, 0.1, 200.0) # fov definido pelo scroll do mouse (45 padrao)
    mat_projection = np.array(mat_projection)    
    return mat_projection

# Definição do fundo da cena (Skybox)

In [6]:

skybox_faces = [
    'obj/skybox/ny.png',  # tras
    'obj/skybox/ny.png',  # frente
    'obj/skybox/nz.png',   # esquerda
    'obj/skybox/nz.png',   # direita
    'obj/skybox/cor.png',  # cima
    'obj/skybox/nz.png'   # baixo
]

# Vertices do cubo do skybox (X e Z = +/- 15, Y = +/- 100)
skybox_vertices = np.array([
    # Face frontal (Z = +15)
    -15.0, -100.0,  15.0,  0.0, 0.0,
     15.0, -100.0,  15.0,  1.0, 0.0,
     15.0,  100.0,  15.0,  1.0, 1.0,
     15.0,  100.0,  15.0,  1.0, 1.0,
    -15.0,  100.0,  15.0,  0.0, 1.0,
    -15.0, -100.0,  15.0,  0.0, 0.0,
    
    # Face traseira (Z = -15)
    -15.0, -100.0, -15.0,  1.0, 0.0,
     15.0, -100.0, -15.0,  0.0, 0.0,
     15.0,  100.0, -15.0,  0.0, 1.0,
     15.0,  100.0, -15.0,  0.0, 1.0,
    -15.0,  100.0, -15.0,  1.0, 1.0,
    -15.0, -100.0, -15.0,  1.0, 0.0,
    
    # Face esquerda (X = -15)
    -15.0,  100.0,  15.0,  1.0, 0.0,
    -15.0,  100.0, -15.0,  0.0, 0.0,
    -15.0, -100.0, -15.0,  0.0, 1.0,
    -15.0, -100.0, -15.0,  0.0, 1.0,
    -15.0, -100.0,  15.0,  1.0, 1.0,
    -15.0,  100.0,  15.0,  1.0, 0.0,
    
    # Face direita (X = +15)
     15.0,  100.0,  15.0,  0.0, 0.0,
     15.0,  100.0, -15.0,  1.0, 0.0,
     15.0, -100.0, -15.0,  1.0, 1.0,
     15.0, -100.0, -15.0,  1.0, 1.0,
     15.0, -100.0,  15.0,  0.0, 1.0,
     15.0,  100.0,  15.0,  0.0, 0.0,
    
    # Face superior (Y = +100)
    -15.0,  100.0, -15.0,  0.0, 1.0,
     15.0,  100.0, -15.0,  1.0, 1.0,
     15.0,  100.0,  15.0,  1.0, 0.0,
     15.0,  100.0,  15.0,  1.0, 0.0,
    -15.0,  100.0,  15.0,  0.0, 0.0,
    -15.0,  100.0, -15.0,  0.0, 1.0,
    
    # Face inferior (Y = -100)
    -15.0, -100.0, -15.0,  0.0, 0.0,
     15.0, -100.0, -15.0,  1.0, 0.0,
     15.0, -100.0,  15.0,  1.0, 1.0,
     15.0, -100.0,  15.0,  1.0, 1.0,
    -15.0, -100.0,  15.0,  0.0, 1.0,
    -15.0, -100.0, -15.0,  0.0, 0.0
], dtype=np.float32)

# Configuração do VAO/VBO do skybox
skybox_vao = glGenVertexArrays(1)
skybox_vbo = glGenBuffers(1)

glBindVertexArray(skybox_vao)
glBindBuffer(GL_ARRAY_BUFFER, skybox_vbo)
glBufferData(GL_ARRAY_BUFFER, skybox_vertices.nbytes, skybox_vertices, GL_STATIC_DRAW)

# Posições
glEnableVertexAttribArray(0)
glVertexAttribPointer(0, 3, GL_FLOAT, GL_FALSE, 5 * sizeof(GLfloat), None)

# Coordenadas de textura
glEnableVertexAttribArray(1)
glVertexAttribPointer(1, 2, GL_FLOAT, GL_FALSE, 5 * sizeof(GLfloat), ctypes.c_void_p(3 * sizeof(GLfloat)))

glBindVertexArray(0)


# Carrega texturas do ceu
def load_skybox_textures(faces):
    textures = []
    for face in faces:
        texture_id = glGenTextures(1)
        glBindTexture(GL_TEXTURE_2D, texture_id)
        
        img = Image.open(face)
        img_data = img.convert("RGB").tobytes("raw", "RGB", 0, -1)
        
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img.size[0], img.size[1], 
                    0, GL_RGB, GL_UNSIGNED_BYTE, img_data)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE)
        
        textures.append(texture_id)
    return textures

skybox_textures = load_skybox_textures(skybox_faces)

# Desenha o ceu
def desenha_skybox():
    glDepthMask(GL_FALSE)  # Desabilita o depth buffer
    
    glBindVertexArray(skybox_vao)
    
    # Desenha cada face com sua textura correspondente
    for i in range(6):
        glBindTexture(GL_TEXTURE_2D, skybox_textures[i])
        glDrawArrays(GL_TRIANGLES, i*6, 6)
    
    glBindVertexArray(0)
    glDepthMask(GL_TRUE)  # Reabilita o depth buffer


# Configuração do Vertex Buffer Object (VBO) para enviar os dados dos vértices e texturas para a GPU

In [7]:
# Gerar os buffers de vértices e coordenadas de textura
buffer_VBO = glGenBuffers(2)

# Carregar os vértices
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

# Upload dos dados de vértices para o buffer
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)

# Corrigir o cálculo do stride para garantir que seja o tamanho do tipo de dados completo
stride_vertices = vertices.itemsize  # Tamanho total de um vértice (no caso, 3 floats)
offset_vertices = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride_vertices, offset_vertices)

# Carregar as coordenadas de textura
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)])  # 2 coordenadas
textures['position'] = textures_coord_list

# Upload dos dados de coordenadas de textura para o buffer
glBindBuffer(GL_ARRAY_BUFFER, buffer_VBO[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)

# Corrigir o cálculo do stride para as coordenadas de textura
stride_textures = textures.itemsize  # Tamanho total de uma coordenada de textura (no caso, 2 floats)
offset_textures = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride_textures, offset_textures)

# Funções de Manipulação de Entrada: Teclado, Mouse e Scroll

In [8]:

# Função de callback para eventos de teclado
def key_event(window, key, scancode, action, mods):
    global cameraPos, cameraFront, cameraUp, s_inc, r_inc, obj_pos_z, obj_pos_x

    # Fecha a janela ao pressionar ESC
    if key == glfw.KEY_ESCAPE and action == glfw.PRESS:
        glfw.set_window_should_close(window, True)
    
    # Velocidade da câmera depende do tempo de frame
    cameraSpeed = 50 * deltaTime

    # Movimentação da câmera com W, A, S, D
    if key == glfw.KEY_W and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos += cameraSpeed * cameraFront
    if key == glfw.KEY_S and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos -= cameraSpeed * cameraFront
    if key == glfw.KEY_A and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
    if key == glfw.KEY_D and (action == glfw.PRESS or action == glfw.REPEAT):
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

    # Limita a posição da câmera dentro de uma região
    cameraPos.x = max(-45.0, min(45.0, cameraPos.x))
    cameraPos.y = max(1.0,    min(9.74, cameraPos.y))
    cameraPos.z = max(-45.0, min(45.0, cameraPos.z))

    # Translação do objeto (humano) com setas direcionais
    if key == glfw.KEY_UP and (action == glfw.PRESS or action == glfw.REPEAT):
        obj_pos_x = min(obj_pos_x + moveSpeed, 2.6)
    if key == glfw.KEY_DOWN and (action == glfw.PRESS or action == glfw.REPEAT):
        obj_pos_x = max(obj_pos_x - moveSpeed, -4.6)
    if key == glfw.KEY_LEFT and (action == glfw.PRESS or action == glfw.REPEAT):
        obj_pos_z = max(obj_pos_z - moveSpeed, -10.0)
    if key == glfw.KEY_RIGHT and (action == glfw.PRESS or action == glfw.REPEAT):
        obj_pos_z = min(obj_pos_z + moveSpeed, -0.4)

    # Escala do objeto (Flor) com teclas Z (aumentar) e X (diminuir)
    if key == glfw.KEY_Z and (action == glfw.PRESS or action == glfw.REPEAT):
        s_inc = min(s_inc + 0.1, 4)
    if key == glfw.KEY_X and (action == glfw.PRESS or action == glfw.REPEAT):
        s_inc = max(s_inc - 0.1, 0.5)

    # Rotação do objeto (Caracol) com teclas F (sentido horário) e G (sentido anti-horário)
    if key == glfw.KEY_F and (action == glfw.PRESS or action == glfw.REPEAT):
        r_inc += 2
    if key == glfw.KEY_G and (action == glfw.PRESS or action == glfw.REPEAT):
        r_inc -= 2

    # Rotação e Escala ao mesmo tempo com a tecla T
    if key == glfw.KEY_T and (action == glfw.PRESS or action == glfw.REPEAT):
        r_inc += 2
        if s_inc < 4: 
            s_inc = s_inc + 0.1
        else:
            s_inc = 0.5

    # Alterna para modo de visualização em wireframe (malha) com a tecla P
    if glfw.KEY_P and (action == glfw.PRESS or action == glfw.REPEAT):
        glPolygonMode(GL_FRONT_AND_BACK, GL_LINE) 
    
    # Alterna para modo de preenchimento padrão com a tecla O
    if glfw.KEY_O and (action == glfw.PRESS or action == glfw.REPEAT):
        glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)


# Callback para ajuste do viewport quando a janela for redimensionada
def framebuffer_size_callback(window, largura, altura):
    glViewport(0, 0, largura, altura)


# Callback para movimentação do mouse (controle da câmera)
def mouse_callback(window, xpos, ypos):
    global cameraFront, lastX, lastY, firstMouse, yaw, pitch

    # Inicializa última posição do mouse
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    # Calcula deslocamentos do mouse
    xoffset = xpos - lastX
    yoffset = lastY - ypos  # y invertido
    lastX = xpos
    lastY = ypos

    # Sensibilidade do mouse
    sensitivity = 0.1
    xoffset *= sensitivity
    yoffset *= sensitivity

    # Atualiza ângulos de direção
    yaw += xoffset
    pitch += yoffset

    # Limita o ângulo de pitch
    pitch = max(-89.0, min(89.0, pitch))

    # Atualiza direção da câmera
    front = glm.vec3()
    front.x = glm.cos(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    front.y = glm.sin(glm.radians(pitch))
    front.z = glm.sin(glm.radians(yaw)) * glm.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


# Callback para o scroll do mouse (ajuste do campo de visão / zoom)
def scroll_callback(window, xoffset, yoffset):
    global fov
    fov -= yoffset
    fov = max(1.0, min(90.0, fov))  # Limita FOV entre 1 e 90 graus


# Registra os callbacks nas funções do GLFW
glfw.set_key_callback(window, key_event)
glfw.set_framebuffer_size_callback(window, framebuffer_size_callback)
glfw.set_cursor_pos_callback(window, mouse_callback)
glfw.set_scroll_callback(window, scroll_callback)

# Captura e desabilita o cursor do mouse (para controle de câmera)
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED)

# Função Principal de Loop

In [9]:
glfw.show_window(window)
glEnable(GL_DEPTH_TEST)

while not glfw.window_should_close(window):

    currentFrame = glfw.get_time()
    deltaTime = currentFrame - lastFrame
    lastFrame = currentFrame

    glfw.poll_events() 
       
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    

    # Desenha o fundo da cena
    desenha_skybox() 

    # Desenho dos objetos
    desenha_sofa()
    desenha_flor(s_inc)
    desenha_fridge()
    desenha_tv()
    desenha_table()
    desenha_bread()
    desenha_snail(r_inc)
    desenha_basket()
    desenha_tree() 
    desenha_man(obj_pos_z, obj_pos_x)
    desenha_table_in()
    desenha_house()
    desenha_grama()
   

    glfw.swap_buffers(window)

glfw.terminate()